In [157]:
import pandas as pd
import csv
import numpy as np
np.random.seed(357823)
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import confusion_matrix, recall_score



In [158]:
# Lectura del csv para la tabla con todas las métricas
    #nodes = pd.read_csv("../tablas/tableWithAllAtributes.csv")

# Lectura del csv para la tabla con todos los atributos sin los de agrupamiento
nodesWithoutClustering = pd.read_csv("../tablas/tableWithoutClustering.csv")

# Lectura del csv para la tabla con todos los atributos sin los de comunidades
    #nodesWithoutCommunity = pd.read_csv("../tablas/tableWithoutCommunity.csv")

#Tablas con todos los atributos sin los de nucleos
    #nodesWithoutKernel = pd.read_csv("../tablas/tableWithoutKernel.csv")





In [159]:
nodesWithoutClustering.head()

,id_node,name,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,Eiryyy,0.0,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.0
1,1,shawflying,0.0,0.000212,0.294956,1.149733e-06,0.002227,0.151515,0.0
2,2,JpMCarrilho,1.0,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.0
3,3,SuhwanCha,0.0,0.000133,0.278718,5.316292e-05,0.004454,0.090909,0.0
4,4,sunilangadi2,1.0,0.000053,0.243084,6.134318e-09,0.011136,0.030303,0.0


In [160]:
# PREPARACIÓN DE LOS DATOS PARA LA TABLA CON TODOS LOS ATRIBUTOS SIN LOS DE AGRUPAMIENTO PARA EL MODELO DE ENTRENAMIENTO
atributos_discretos = ['name']
atributos_continuos = ['id_node', 'ml_target', 'degree_centrality','closeness_centrality','betweenness_centrality','greedy_modularity_communities','Core number','asyn_lpa_communities']
atributos = nodesWithoutClustering.loc[:, atributos_discretos + atributos_continuos]
atributos.head()

,name,id_node,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,greedy_modularity_communities,Core number,asyn_lpa_communities
0,Eiryyy,0,0.0,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.0
1,shawflying,1,0.0,0.000212,0.294956,1.149733e-06,0.002227,0.151515,0.0
2,JpMCarrilho,2,1.0,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.0
3,SuhwanCha,3,0.0,0.000133,0.278718,5.316292e-05,0.004454,0.090909,0.0
4,sunilangadi2,4,1.0,0.000053,0.243084,6.134318e-09,0.011136,0.030303,0.0


In [161]:
# Objetivo
objetivo = nodesWithoutClustering['ml_target']
objetivo.head() 

0    0.0
1    0.0
2    1.0
3    0.0
4    1.0
Name: ml_target, dtype: float64

In [162]:
# Codificación
codificador_atributos_discretos = OrdinalEncoder() # Crear una instancia de la clase correspondiente
codificador_atributos_discretos.fit(atributos[atributos_discretos]) # Usar el método fit para ajustar a los datos los parámetros de la codificación

# Vemos información sobre los atributos discretos
print("///Información sobre los atributos discretos///")
print('Número de atributos detectados:',
      f'{codificador_atributos_discretos.n_features_in_}')
print()
print('Nombres de los atributos detectados:')
print(f'{codificador_atributos_discretos.feature_names_in_}')
print()
print('Categorías detectadas de cada atributo:')
for atributo, categorías in zip(
    codificador_atributos_discretos.feature_names_in_,
    codificador_atributos_discretos.categories_):
    print(f'{atributo}: {categorías}')

# Ahora aplicamos el método transform para codificar los datos
atributos[atributos_discretos] = codificador_atributos_discretos.transform(
    atributos[atributos_discretos]
)
print()
print("///Tabla con la columna name cambiada a números///")
atributos.head()


///Información sobre los atributos discretos///
Número de atributos detectados: 1

Nombres de los atributos detectados:
['name']

Categorías detectadas de cada atributo:
name: ['007arunwilson' '007jedgar' '00Kai0' ... 'timothykimemia' 'timoxley'
 'timqian']

///Tabla con la columna name cambiada a números///


,name,id_node,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,greedy_modularity_communities,Core number,asyn_lpa_communities
0,2325.0,0,0.0,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.0
1,35055.0,1,0.0,0.000212,0.294956,1.149733e-06,0.002227,0.151515,0.0
2,4022.0,2,1.0,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.0
3,7220.0,3,0.0,0.000133,0.278718,5.316292e-05,0.004454,0.090909,0.0
4,36547.0,4,1.0,0.000053,0.243084,6.134318e-09,0.011136,0.030303,0.0


In [163]:
# Ahora (aunque ya están en tipo integer) voy a transformar por si acaso a números la variable objetivo 
print("///Variable objetivo de tipo numérico///")
codificador_objetivo = LabelEncoder()
objetivo = codificador_objetivo.fit_transform(objetivo)
print(f'Clases detectadas: {codificador_objetivo.classes_}')
objetivo

///Variable objetivo de tipo numérico///
Clases detectadas: [0. 1.]


array([0, 0, 1, ..., 0, 1, 0], dtype=int64)

In [164]:
# Entrenamiento del modelo de Naive Bayes para la tabla con todos los atributos sin los de agrupamiento

In [165]:
# Discretizamos:
# Discretizamos usando uniform ya que si usamos el que nos da la practica que es la estrategia de quantile se eliminan datos por ser los intervalos demasiado pequeños
discretizador = KBinsDiscretizer(
    n_bins=3,  # Reducimos el número de bins a 3
    encode='ordinal',  # Los intervalos se codifican numéricamente
    strategy='uniform'  # Intervalos de igual tamaño
    )

# Como nos interesa conservar los atributos continuos originales, realizamos
# la discretización sobre una copia del DataFrame de atributos
atributos_discretizados = atributos.copy()
atributos_discretizados[atributos_continuos] = discretizador.fit_transform(
    atributos_discretizados[atributos_continuos]
)
atributos_discretizados.head()


### REVISAR DEBIDO A QUE NO SE DISCRETIZA EL NAME Y NO SE SI HAY QUE DISCRETIZAR TAMBIÉN ID NODE O NO ###

,name,id_node,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,greedy_modularity_communities,Core number,asyn_lpa_communities
0,2325.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,35055.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,4022.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7220.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,36547.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0


In [166]:
# Naive Bayes con suavizado de Laplace
clasificador_NB = CategoricalNB(alpha=1)  # alpha es el parámetro de suavizado
clasificador_NB.fit(atributos_discretizados, objetivo)

CategoricalNB(alpha=1)

In [167]:
# Podemos comprobar cómo se han estimado los parámetros del modelo mediante una estimación de máxima verosimilitud usando suavizado de Laplace
for clase, cantidad_ejemplos_clase, log_probabilidad_clase in zip(
    clasificador_NB.classes_,
    clasificador_NB.class_count_,
    clasificador_NB.class_log_prior_
):
        print(f'Cantidad de ejemplos para la clase {clase}: {cantidad_ejemplos_clase}')
        print(f'Logaritmo de la probabilidad aprendida para la clase {clase}: {log_probabilidad_clase}')

Cantidad de ejemplos para la clase 0: 27961.0
Logaritmo de la probabilidad aprendida para la clase 0: -0.2988494123488099
Cantidad de ejemplos para la clase 1: 9739.0
Logaritmo de la probabilidad aprendida para la clase 1: -1.353521651474903


In [168]:
for i, atributo in enumerate(clasificador_NB.feature_names_in_):
    print(f'Cantidad de ejemplos para el atributo {atributo}:')
    print(pd.DataFrame(clasificador_NB.category_count_[i],
                       columns=range(clasificador_NB.n_categories_[i]),
                       index=clasificador_NB.classes_))
    print(f'Logaritmos de las probabilidades aprendidas para el atributo {atributo}:')
    print(pd.DataFrame(clasificador_NB.feature_log_prob_[i],
                       columns=range(clasificador_NB.n_categories_[i]),
                       index=clasificador_NB.classes_))
    print()

Cantidad de ejemplos para el atributo name:
   0      1      2      3      4      5      6      7      8      9      ...  \
0    1.0    1.0    0.0    1.0    0.0    1.0    1.0    1.0    1.0    1.0  ...   
1    0.0    0.0    1.0    0.0    1.0    0.0    0.0    0.0    0.0    0.0  ...   

   37690  37691  37692  37693  37694  37695  37696  37697  37698  37699  
0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0  
1    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  

[2 rows x 37700 columns]
Logaritmos de las probabilidades aprendidas para el atributo name:
       0          1          2          3          4          5      \
0 -10.399113 -10.399113 -11.092260 -10.399113 -11.092260 -10.399113   
1 -10.767200 -10.767200 -10.074053 -10.767200 -10.074053 -10.767200   

       6          7          8          9      ...      37690      37691  \
0 -10.399113 -10.399113 -10.399113 -10.399113  ... -10.399113 -10.399113   
1 -10.767200 -10.767200 -10.767200

In [169]:
predicciones = clasificador_NB.predict(atributos_discretizados)
predicciones

array([0, 0, 1, ..., 0, 1, 0], dtype=int64)

In [170]:
confusion_matrix(objetivo, predicciones)

array([[27961,     0],
       [    0,  9739]], dtype=int64)

In [171]:
recall_score(objetivo, predicciones)

1.0